In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
import sys
import matplotlib.pyplot as plt

# --- PATH SETUP ---
# Links the notebook to your 'src' folder
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# --- IMPORTS ---
try:
    from src.hqcnn_model import HQCNN
    from src.data_loader import get_data_loaders
    print("✅ Infrastructure: Connected to 'src' successfully.")
except ImportError as e:
    print(f"❌ Error: {e}")
    print("Stop! Please ensure your src/ folder files are populated.")

# --- GPU CHECK ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 System Check: Training on {device}")
if device.type == 'cuda':
    print(f"   GPU Model: {torch.cuda.get_device_name(0)}")

✅ Infrastructure: Connected to 'src' successfully.
🚀 System Check: Training on cuda
   GPU Model: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader

# --- 1. HYPERPARAMETERS (Optimized for 10% Data) ---
BATCH_SIZE = 50        
EPOCHS = 35            
LEARNING_RATE = 0.001  
DATA_DIR = '../data/GTSRB'

# --- 2. LOAD FULL DATA ---
print("⏳ Loading Full Dataset (to be decimated)...")
full_train_loader, full_val_loader = get_data_loaders(DATA_DIR, batch_size=BATCH_SIZE)

# --- 3. ✂️ THE 10% DECIMATION ---
def get_subset(loader, fraction=0.10):
    dataset = loader.dataset
    num_samples = len(dataset)
    split = int(np.floor(fraction * num_samples))
    
    # Random Seed 42 = Reproducible Science for your Paper
    indices = list(range(num_samples))
    np.random.seed(42) 
    np.random.shuffle(indices)
    
    # Slice the top 10%
    subset_indices = indices[:split]
    
    return Subset(dataset, subset_indices)

print(f"\n✂️ Slicing Data to 10%...")
train_subset = get_subset(full_train_loader, fraction=0.10)
val_subset = get_subset(full_val_loader, fraction=0.10)

# Create NEW DataLoaders
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# --- 4. VERIFICATION ---
print("-" * 40)
print(f"✅ RESEARCH MODE: 10% DATA ACTIVATED")
print("-" * 40)
print(f"   Training Images:  {len(train_loader.dataset)} (Speed: ~25 mins/epoch)")
print(f"   Validation Images: {len(val_loader.dataset)}")
print("-" * 40)

# --- 5. INITIALIZE MODEL ---
model = HQCNN(n_classes=43).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print("🧠 HQCNN Model Initialized.") 

⏳ Loading Full Dataset (to be decimated)...
Dataset Loaded: 33327 Train | 5882 Val

✂️ Slicing Data to 10%...
----------------------------------------
✅ RESEARCH MODE: 10% DATA ACTIVATED
----------------------------------------
   Training Images:  3332 (Speed: ~25 mins/epoch)
   Validation Images: 588
----------------------------------------
🧠 HQCNN Model Initialized.


In [3]:
def ignite_training(num_epochs):
    print(f"\n🔥 IGNITION: Starting Full Hybrid Training ({num_epochs} Epochs)...")
    start_time = time.time()
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_acc = 0.0
    total_batches = len(train_loader)

    for epoch in range(num_epochs):
        epoch_start = time.time()
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        print(f"\n--- Epoch {epoch+1}/{num_epochs} ---")
        
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            # Forward Pass (Quantum Simulation happens here)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward Pass
            loss.backward()
            optimizer.step()
            
            # Metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # 💓 HEARTBEAT: Print status every 10 batches (approx 45s)
            # This keeps you informed that the GPU is working
            if (i + 1) % 10 == 0:
                batch_acc = 100 * (predicted == labels).sum().item() / labels.size(0)
                print(f"   Batch [{i+1}/{total_batches}] | Loss: {loss.item():.4f} | Acc: {batch_acc:.1f}%")

        # --- END OF EPOCH ---
        avg_train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        
        # Validation Phase
        print("   ⏳ Validating...")
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss_accum = 0.0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss_accum += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss_accum / len(val_loader)
        
        epoch_time = time.time() - epoch_start
        print(f"✅ Epoch {epoch+1} Complete | Time: {epoch_time/60:.1f} min")
        print(f"   Results: Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}% | Val Loss: {avg_val_loss:.4f}")
        
        # Store History
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)
        
        # Save Best Model (Critical for hitting 99.63%)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), '../models/best_hqcnn.pth')
            print(f"   💾 New Best Model Saved ({val_acc:.2f}%)")

    total_time = time.time() - start_time
    print(f"\n🏆 Training Complete in {total_time/3600:.2f} hours.")
    print(f"   Best Validation Accuracy: {best_val_acc:.2f}%")
    
    return history

In [4]:
# 1. RUN TRAINING
# This will take time! Monitor the heartbeat messages.
history = ignite_training(EPOCHS)

# 2. PLOT RESULTS
plt.figure(figsize=(14, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history['train_acc'], label='Train Acc', color='blue')
plt.plot(history['val_acc'], label='Val Acc', color='green')
plt.axhline(y=99.63, color='r', linestyle='--', label='Target (99.63%)') # Target Line
plt.title('Accuracy Trajectory')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True, alpha=0.3)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history['train_loss'], label='Train Loss', color='red')
plt.plot(history['val_loss'], label='Val Loss', color='orange')
plt.title('Loss Convergence')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


🔥 IGNITION: Starting Full Hybrid Training (35 Epochs)...

--- Epoch 1/35 ---
   Batch [10/67] | Loss: 3.7331 | Acc: 2.0%
   Batch [20/67] | Loss: 3.7090 | Acc: 2.0%
   Batch [30/67] | Loss: 3.6432 | Acc: 6.0%
   Batch [40/67] | Loss: 3.4896 | Acc: 2.0%
   Batch [50/67] | Loss: 3.5541 | Acc: 6.0%
   Batch [60/67] | Loss: 3.4669 | Acc: 0.0%
   ⏳ Validating...
✅ Epoch 1 Complete | Time: 39.1 min
   Results: Train Acc: 4.62% | Val Acc: 5.61% | Val Loss: 3.4537
   💾 New Best Model Saved (5.61%)

--- Epoch 2/35 ---
   Batch [10/67] | Loss: 3.4228 | Acc: 2.0%
   Batch [20/67] | Loss: 3.2572 | Acc: 0.0%
   Batch [30/67] | Loss: 3.5578 | Acc: 6.0%
   Batch [40/67] | Loss: 3.6139 | Acc: 6.0%
   Batch [50/67] | Loss: 3.3866 | Acc: 12.0%
   Batch [60/67] | Loss: 3.4102 | Acc: 4.0%
   ⏳ Validating...
✅ Epoch 2 Complete | Time: 40.3 min
   Results: Train Acc: 5.52% | Val Acc: 5.61% | Val Loss: 3.4519

--- Epoch 3/35 ---
   Batch [10/67] | Loss: 3.4793 | Acc: 12.0%
   Batch [20/67] | Loss: 3.4189 | 

KeyboardInterrupt: 